In [ ]:
from itertools import product

def shortest_path_ilp_bruteforce(adj, source, target):
    n = len(adj)

    
    edges = []
    for i in range(n):
        for j in range(n):
            if adj[i][j] not in (0, None):
                edges.append((i, j, adj[i][j]))

    print("Total edges:", len(edges))
    print("Trying all ILP combinations...")

    best_cost = float('inf')
    best_choice = None

    
    for choice in product([0,1], repeat=len(edges)):
        flow = [0] * n
        cost = 0

       
        for i in range(len(edges)):
            if choice[i] == 1:
                u, v, w = edges[i]
                flow[u] += 1
                flow[v] -= 1
                cost += w

        # ILP flow constraints
        valid = True
        for i in range(n):
            if i == source and flow[i] != 1:
                valid = False
            elif i == target and flow[i] != -1:
                valid = False
            elif i not in (source, target) and flow[i] != 0:
                valid = False

        if valid:
            if cost < best_cost:
                best_cost = cost
                best_choice = choice

    if best_choice is None:
        print("No path exists.")
        return

    
    path = [source]
    curr = source

    while curr != target:
        for i in range(len(edges)):
            if best_choice[i] == 1:
                u, v, _ = edges[i]
                if u == curr and v not in path:
                    path.append(v)
                    curr = v
                    break

    print("Shortest Cost:", best_cost)
    print("Path:", " -> ".join(map(str, path)))


In [17]:
adj_matrix = [
    [0, 3, 1, 0],
    [2, 1, 100, 6],
    [1, 0, 100, 2],
    [1, 3, 0, 0]
]

shortest_path_ilp_bruteforce(adj_matrix, source=3, target=2)


Total edges: 11
Trying all ILP combinations...
Shortest Cost: 2
Path: 3 -> 0 -> 2


In [ ]:
import math

class BranchAndBoundILP:
    def __init__(self, adj, source, target):
        self.adj = adj
        self.n = len(adj)
        self.source = source
        self.target = target
        self.edges = []
        self.best_cost = math.inf
        self.best_solution = None
        
        for i in range(self.n):
            for j in range(self.n):
                if adj[i][j] not in (0, None):
                    self.edges.append((i, j, adj[i][j]))

    def check_constraints(self, solution):
        flow = [0]*self.n
        cost = 0

        for i in range(len(self.edges)):
            if solution[i] == 1:
                u, v, w = self.edges[i]
                flow[u] += 1
                flow[v] -= 1
                cost += w

        for i in range(self.n):
            if i == self.source and flow[i] != 1:
                return False, None
            elif i == self.target and flow[i] != -1:
                return False, None
            elif i not in (self.source,self.target) and flow[i] != 0:
                return False, None

        return True, cost

    def compute_lower_bound(self, solution):
        cost = 0
        for i in range(len(self.edges)):
            if solution[i] == 1:
                cost += self.edges[i][2]
        return cost

    
    def branch_and_bound(self, solution, idx):
        
        bound = self.compute_lower_bound(solution)
        if bound >= self.best_cost:
            return  

       
        if idx == len(solution):
            valid, cost = self.check_constraints(solution)
            if valid and cost < self.best_cost:
                self.best_cost = cost
                self.best_solution = solution.copy()
            return

        
        solution[idx] = 1
        self.branch_and_bound(solution, idx+1)

        
        solution[idx] = 0
        self.branch_and_bound(solution, idx+1)

    
    def solve(self):
        solution = [0]*len(self.edges)
        self.branch_and_bound(solution, 0)

        if self.best_solution is None:
            print("No path exists.")
            return

        
        path = [self.source]
        curr = self.source
        while curr != self.target:
            for i, use in enumerate(self.best_solution):
                if use == 1:
                    u, v, _ = self.edges[i]
                    if u == curr and v not in path:
                        path.append(v)
                        curr = v
                        break

        print("Shortest Path:", " -> ".join(map(str,path)))
        print("Cost:", self.best_cost)



adj = [
    [0, 3, 1, 0],
    [2, 1, 100, 6],
    [1, 0, 100, 2],
    [1, 3, 0, 0]
]

solver = BranchAndBoundILP(adj, source=3, target=2)
solver.solve()


Shortest Path: 3 -> 0 -> 2
Cost: 2
